In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
import pandas as pd
import numpy as np

naver_data = pd.read_table('./data/ratings.txt')
print(naver_data)

              id                                           document  label
0        8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1        8132799  디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...      1
2        4655635               폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.      1
3        9251303  와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...      1
4       10067386                        안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.      1
...          ...                                                ...    ...
199995   8963373                                     포켓 몬스터 짜가 ㅡㅡ;;      0
199996   3302770                                              쓰.레.기      0
199997   5458175                  완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.      0
199998   6908648                왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ      0
199999   8548411                                    포풍저그가나가신다영차영차영차      0

[200000 rows x 3 columns]


In [3]:
# 정규표현식을 사용하여 단어가 아닌것을 삭제

naver_data['document'] = naver_data['document'].str.replace("[^\w]|br", " ")
print(naver_data)

              id                                           document  label
0        8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1        8132799  디자인을 배우는 학생으로  외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...      1
2        4655635               폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음   최고       1
3        9251303  와   연기가 진짜 개쩔구나   지루할거라고 생각했는데 몰입해서 봤다   그래 이런...      1
4       10067386                        안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화       1
...          ...                                                ...    ...
199995   8963373                                     포켓 몬스터 짜가 ㅡㅡ        0
199996   3302770                                              쓰 레 기      0
199997   5458175                  완전 사이코영화  마지막은 더욱더 이 영화의질을 떨어트린다       0
199998   6908648                왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ      0
199999   8548411                                    포풍저그가나가신다영차영차영차      0

[200000 rows x 3 columns]


In [4]:
from sklearn.model_selection import train_test_split

document_train, document_test, y_train, y_test = train_test_split(naver_data['document'], naver_data['label'], test_size = 0.25, shuffle=True, random_state=3)
document_train.shape, document_test.shape, y_train.shape, y_test.shape

((150000,), (50000,), (150000,), (50000,))

In [5]:
from konlpy.tag import Okt

# 예시용 stopwords (불용어)
stopwords = ['을', '를', '이', '가', '으로', '에게']

# 토큰화 진행

X_train = []
for stc in document_train:
    token = []
    words = Okt().morphs(stc)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

# X_test = []
# for stc in review_test:
#     token = []
#     words = Okt().morphs(stc)
#     for word in words:
#         if word not in stopwords:
#             token.append(word)
#     X_test.append(token)

TypeError: No matching overloads found for kr.lucypark.okt.OktInterface.tokenize(float,java.lang.Boolean,java.lang.Boolean), options are:
	public java.util.List kr.lucypark.okt.OktInterface.tokenize(java.lang.String,java.lang.Boolean,java.lang.Boolean)

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:142)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# fit on texts는 train기준으로 한번만 한다. 
# train에 해당되지 않는 단어는 사라진다.
# 데이터가 사라지는게 싫으면 전체데이터를 stopwords한 다음 fit_on_text를 진행한다.
# 안에 값을 넣어주면 해당되는 값만큼 인덱스를 부여한다.
# 무의미하게 아무렇게 단어를 고르는게 아니라 단어 빈도수가 너무 적은 필요없는 단어를 제외하고 필요한 단어 5000개만 잡아서 넣어준다.
# 데이터를 분석하여 유의미한 데이터 갯수를 파악한 뒤어 사용해야한다. 그냥 사용하다가는 의미있는 데이터가 날라간다.

tokenizer = Tokenizer(5000) 
tokenizer.fit_on_texts(X_train)

# 동일한 단어는 동일한 인덱스
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)